In [6]:
import re

In [7]:
corpus_name = 'en_ewt-ud-train.conllu'
with open(corpus_name) as f:
    corpus = []
    #corpus  = re.findall(r"^\d\t(.*)", data)
    for line in f.readlines():
        if re.match(r"^\d\t(.*)", line) != None:
            line = re.split(r'\t', line)
            corpus.append(line)
print('Corpus length (in words) = %d' % len(corpus))

Corpus length (in words) = 95250


Пусть первые 7000 слов будут обучающей выборкой, а остальные контрольной

In [8]:
test = corpus[7002:]
corpus = corpus[:7002]

##  HMM-tagger

In [9]:
tags = {}
prob_tags = {}
for token in range(len(corpus)):
    tag = corpus[token][4]
    counter = tags.get(tag)
    if counter == None:
        counter = 0
    tags.update({tag:counter+1})
for tag in tags:
    tag_count = tags.get(tag)
    count = 1 + tag_count/len(corpus)
    prob_tags.update({tag:count})

In [10]:
tag_word_bag = {}
word_tag_bag = {}
for token in range(len(corpus)):
    pair = (corpus[token][4], corpus[token][1])
    counter = tag_word_bag.get(pair)
    if counter == None:
        counter = 0
    tag_word_bag.update({pair:counter+1})

for thing in tag_word_bag:
    tag = thing[0]
    word = thing[1]
    count = 1 + tag_word_bag.get(thing)/tags.get(tag)
    word_tag_bag.setdefault(word, {})[tag] = count

In [20]:
tag_bigram_ = {}
tag_bigram = {}
for thing in range(len(corpus)-1):
    pair = (corpus[thing][4], corpus[thing+1][4])
    count = tag_bigram_.get(pair)
    if count == None:
        count = 0
    tag_bigram_.update({pair:count+1})

for thing in tag_bigram_:
    tag_1 = thing[0]
    tag_2 = thing[1]
    count = 1 + tag_bigram_.get(thing)/len(tag_bigram_)
    tag_bigram.setdefault(tag_1, {})[tag_2] = count

In [72]:
tagged_seq = []
for word in range(len(test)):
    prob = []
    if test[word][0] == str(1):
        tags = word_tag_bag.get(test[word][1])
        all_tags = []
        try:    
            for thing in tags:   
                tag_prob = []
                prob = tags.get(thing)
                tag_prob.append(prob)
                tag_prob.append(thing)
                all_tags.append(tag_prob)
        except TypeError:
            all_tags.append([1, 'UnkWord'])
        all_tags = sorted(all_tags, reverse = True)
        word_and_the_tag = [test[word][1], all_tags[0][1], all_tags[0][0]]
        tagged_seq.append(word_and_the_tag)
    else:
        print (tagged_seq)
        all_tags = []
        prev_tag = tagged_seq[word-1][1]
        prev_tags = tag_bigram.get(prev_tag)
        em_tags = word_tag_bag.get(test[word][1])
        print(em_tags)
        print(test[word][1])
        for thing in em_tags:
            print (thing)
            if prev_tags.get(thing) == None:
                prob == prev_tags.get(thing)
            else:
                prob = prev_tags.get(thing) * em_tags.get(thing)
            all_tags.append([thing, prob])
        for thing in prev_tags:
            if thing == None:
                prob = em_tags.get(thing)
            all_tags.append([thing, prob])
        all_tags = sorted(all_tags, reverse = True)
        word_and_the_tag = [test[word][1], all_tags[0][1], all_tags[0][0]]
        tagged_seq.append(word_and_the_tag)
        print (all_tags)
print (tagged_seq)

[['Police', 'NNS', 1.0035211267605635]]
{'CC': 1.5864197530864197}
and
CC
[['WP', 1.621673525377229], ['WDT', 1.621673525377229], ['VBZ', 1.621673525377229], ['VBP', 1.621673525377229], ['VBN', 1.621673525377229], ['VBG', 1.621673525377229], ['VBD', 1.621673525377229], ['TO', 1.621673525377229], ['RP', 1.621673525377229], ['RB', 1.621673525377229], ['PRP$', 1.621673525377229], ['PRP', 1.621673525377229], ['POS', 1.621673525377229], ['NNPS', 1.621673525377229], ['NNP', 1.621673525377229], ['NN', 1.621673525377229], ['MD', 1.621673525377229], ['JJ', 1.621673525377229], ['IN', 1.621673525377229], ['HYPH', 1.621673525377229], ['EX', 1.621673525377229], ['DT', 1.621673525377229], ['CC', 1.621673525377229], ['CC', 1.621673525377229], [':', 1.621673525377229], ['.', 1.621673525377229], ['-LRB-', 1.621673525377229], [',', 1.621673525377229], ["''", 1.621673525377229]]
[['Police', 'NNS', 1.0035211267605635], ['and', 1.621673525377229, 'WP']]
{'NNS': 1.0035211267605635}
prosecutors
NNS


AttributeError: 'NoneType' object has no attribute 'get'

In [70]:
word_tag_bag.get('prosecutors')

{'NNS': 1.0035211267605635}

In [71]:
tag_bigram.get('NNS')

{'VBD': 1.0414814814814815,
 'DT': 1.0103703703703704,
 'NNP': 1.0118518518518518,
 'IN': 1.1377777777777778,
 'VBP': 1.0577777777777777,
 'NNPS': 1.002962962962963,
 'EX': 1.0014814814814814,
 'PRP': 1.0074074074074073,
 "''": 1.002962962962963,
 'JJ': 1.0103703703703704,
 'RB': 1.0162962962962963,
 ':': 1.0014814814814814,
 ',': 1.0340740740740741,
 'CC': 1.0222222222222221,
 'WDT': 1.0103703703703704,
 '.': 1.0118518518518518,
 'MD': 1.005925925925926,
 'NN': 1.005925925925926,
 'TO': 1.0044444444444445,
 'VBN': 1.0074074074074073,
 '-LRB-': 1.0014814814814814,
 'VBG': 1.002962962962963,
 'VBZ': 1.002962962962963,
 'WP': 1.002962962962963,
 'POS': 1.0014814814814814,
 'HYPH': 1.0014814814814814,
 'RP': 1.0014814814814814,
 'PRP$': 1.0014814814814814}